In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install pmdarima --quiet
import pmdarima as pm
from datetime import date, timedelta, datetime

In [4]:
# load the data
data = pd.read_csv('Nat_Gas.csv')

In [5]:
# preprocess the data
data['Dates'] = pd.to_datetime(data['Dates'])
prices = data['Prices'].values
dates = data['Dates'].values
data = data.set_index(['Dates'])
daily_data = data.resample('D').ffill()
# Reset the index to include all daily dates
daily_data.reset_index(inplace=True)
daily_data = daily_data.set_index(['Dates'])

In [6]:
def sarima_forecast(data, input_date_str, periods=24):
    # Fit a SARIMA model using auto_arima
    SARIMA_model = pm.auto_arima(data, start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0,
                             seasonal=True, d=None, D=1, trace=False, error_action='ignore', suppress_warnings=True, stepwise=True)

    # Parse the input date
    input_date = pd.to_datetime(input_date_str)

    # Forecast for the input date and specified number of periods
    forecast = SARIMA_model.predict(n_periods=periods)
    
    # Extract the forecasted price for the input date
    input_date_index = pd.date_range(start=input_date, periods=1, freq='M')
    forecasted_price = forecast[input_date_index[0]]

    return forecasted_price

In [11]:
def calculate_contract_value(data,daily_data,injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_volume, storage_cost):
    total_value = 0

    # Ensure input lists have the same length
    if len(injection_dates) != len(withdrawal_dates):
        raise ValueError("Input lists must have the same length.")

    for i in range(len(injection_dates)):
        injection_date = pd.to_datetime(injection_dates[i])
        withdrawal_date = pd.to_datetime(withdrawal_dates[i])
        if injection_date < pd.to_datetime("2024-09-30"):
            buy_price = daily_data[daily_data.index == injection_date]['Prices'].values[0]
        else:
            buy_price = sarima_forecast(data['Prices'],injection_date)
        if withdrawal_date < pd.to_datetime("2024-09-30"):
            sell_price =  daily_data[daily_data.index == withdrawal_date]['Prices'].values[0]
        else:
            sell_price = sarima_forecast(data['Prices'],withdrawal_date)
        # Calculate the value for each injection/withdrawal cycle
        value = (sell_price - buy_price) * min(max_volume, injection_rate * (withdrawal_date - injection_date).days)
        value -= storage_cost * (withdrawal_date - injection_date).days

        # Add the value to the total
        total_value += value
    return total_value

In [12]:
# Sample inputs:

injection_dates = ["2023-11-30", "2024-10-01"]
withdrawal_dates = ["2025-03-01", "2025-2-01"]
injection_rate = 10000  # MMBtu per day
withdrawal_rate = 10000  # MMBtu per day
max_volume = 10000000  # Maximum storage volume
storage_cost = 100  # Cost per day

contract_value = calculate_contract_value(data,daily_data,injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_volume, storage_cost)
contract_value

5882514.905531632